# Some reference and ideas:
-  I think using KNN to fill empty value is reasonbale.(I see some using 0 to fill in,but I don't feel it's a good choice,let's try KNN first)
- some resouces(try to have a brief look of them):
1.https://github.com/sudhamstarun/Costa-Rican-Household-Poverty-Level-Prediction<br>
2.cornell paper.pdf on the left side<br>


### costa rican household poverty level prediction

# Task 1:fill the data and output into a csv file

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
print(df_train.shape, df_test.shape)

(9557, 143) (23856, 142)


In [ ]:
_deepnote_run_altair(df, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"Id","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"v2a1","type":"quantitative","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

NameError: name 'df' is not defined

In [ ]:
df_train["Target"].value_counts()

4    5996
2    1597
3    1209
1     755
Name: Target, dtype: int64

In [ ]:
codeBook = pd.read_csv("data/codebook.csv")
codeBook

,Variable name,Variable description
0,v2a1,Monthly rent payment
1,hacdor,=1 Overcrowding by bedrooms
2,rooms,number of all rooms in the house
3,hacapo,=1 Overcrowding by rooms
4,v14a,=1 has toilet in the household
...,...,...
136,SQBhogar_nin,hogar_nin squared
137,SQBovercrowding,overcrowding squared
138,SQBdependency,dependency squared
139,SQBmeaned,meaned squared


In [ ]:
missing_variables_train = pd.DataFrame(df_train.isnull().sum()).rename(columns = {0: 'total'}) 
missing_variables_train = missing_variables_train[missing_variables_train.total != 0]
display(missing_variables_train) # checking which feature columns have missing data so we can fill them up accordingly

missing_variables_test = pd.DataFrame(df_test.isnull().sum()).rename(columns = {0: 'total'}) 
missing_variables_test = missing_variables_test[missing_variables_test.total != 0]
display(missing_variables_test) # checking which feature columns have missing data so we can fill them up accordingly

,total
v2a1,6860
v18q1,7342
rez_esc,7928
meaneduc,5
SQBmeaned,5


,total
v2a1,17403
v18q1,18126
rez_esc,19653
meaneduc,31
SQBmeaned,31


In [ ]:
df2_train = df_train[['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned']]
display(df2_train.head()) # I manually selected just the columns that needed to be imputed

df2_test = df_test[['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned']]
display(df2_test.head()) # I manually selected just the columns that needed to be imputed

,v2a1,v18q1,rez_esc,meaneduc,SQBmeaned
0,190000.0,NaN,NaN,10.0,100.0
1,135000.0,1.0,NaN,12.0,144.0
2,NaN,NaN,NaN,11.0,121.0
3,180000.0,1.0,1.0,11.0,121.0
4,180000.0,1.0,NaN,11.0,121.0


,v2a1,v18q1,rez_esc,meaneduc,SQBmeaned
0,NaN,NaN,NaN,16.5,272.25
1,NaN,NaN,NaN,16.5,272.25
2,NaN,NaN,NaN,16.5,272.25
3,NaN,1.0,NaN,16.0,256.00
4,175000.0,1.0,NaN,NaN,NaN


In [ ]:
import numpy as np
from sklearn.impute import KNNImputer

knn = KNNImputer(n_neighbors = 2)
df2_knn_train = knn.fit_transform(df2_train)
df2_knn_test = knn.fit_transform(df2_test)

In [ ]:
print(df2_knn_train) # imputed values for the 5 columns with missing variables
print(df2_knn_test) # imputed values for the 5 columns with missing variables

[[1.90000e+05 1.00000e+00 0.00000e+00 1.00000e+01 1.00000e+02]
 [1.35000e+05 1.00000e+00 5.00000e-01 1.20000e+01 1.44000e+02]
 [1.80000e+05 1.00000e+00 5.00000e-01 1.10000e+01 1.21000e+02]
 ...
 [8.00000e+04 1.00000e+00 0.00000e+00 8.25000e+00 6.80625e+01]
 [8.00000e+04 1.00000e+00 0.00000e+00 8.25000e+00 6.80625e+01]
 [8.00000e+04 1.00000e+00 0.00000e+00 8.25000e+00 6.80625e+01]]
[[3.8300e+05 2.0000e+00 0.0000e+00 1.6500e+01 2.7225e+02]
 [3.8300e+05 2.0000e+00 0.0000e+00 1.6500e+01 2.7225e+02]
 [3.8300e+05 2.0000e+00 0.0000e+00 1.6500e+01 2.7225e+02]
 ...
 [1.2000e+05 1.0000e+00 1.0000e+00 6.0000e+00 3.6000e+01]
 [1.2000e+05 1.0000e+00 0.0000e+00 6.0000e+00 3.6000e+01]
 [1.2000e+05 1.0000e+00 0.0000e+00 6.0000e+00 3.6000e+01]]


In [ ]:
# transferring the imputed values back to df 
df_train['v2a1'] = df2_knn_train[:, 0]
df_train['v18q1'] = df2_knn_train[:, 1]
df_train['rez_esc'] = df2_knn_train[:, 2]
df_train['meaneduc'] = df2_knn_train[:, 3]
df_train['SQBmeaned'] = df2_knn_train[:, 4]

df_test['v2a1'] = df2_knn_test[:, 0]
df_test['v18q1'] = df2_knn_test[:, 1]
df_test['rez_esc'] = df2_knn_test[:, 2]
df_test['meaneduc'] = df2_knn_test[:, 3]
df_test['SQBmeaned'] = df2_knn_test[:, 4]

In [ ]:
display(df_train.head()) # Data with no more missing values
display(df_test.head()) # Data with no more missing values

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,1.0,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,180000.0,0,8,0,1,1,0,1.0,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,ID_2f6873615,383000.0,0,5,0,1,1,0,2.0,1,...,4,0,16,9,0,1,2.25,0.25,272.25000,16
1,ID_1c78846d2,383000.0,0,5,0,1,1,0,2.0,1,...,41,256,1681,9,0,1,2.25,0.25,272.25000,1681
2,ID_e5442cf6a,383000.0,0,5,0,1,1,0,2.0,1,...,41,289,1681,9,0,1,2.25,0.25,272.25000,1681
3,ID_a8db26a79,187500.0,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00000,3481
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,192.22222,324


In [ ]:
df_train.describe()

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
count,9.557000e+03,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,...,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000
mean,1.584290e+05,0.038087,4.955530,0.023648,0.994768,0.957623,0.231767,1.401057,0.385895,1.559171,...,74.222769,1643.774302,19.132887,53.500262,3.844826,3.249485,3.900409,102.597750,1643.774302,3.302292
std,1.044605e+05,0.191417,1.468381,0.151957,0.072145,0.201459,0.421983,0.775818,0.680779,1.036574,...,76.777549,1741.197050,18.751395,78.445804,6.946296,4.129547,12.511831,93.493447,1741.197050,1.009565
min,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,1.000000
25%,1.000000e+05,0.000000,4.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,16.000000,289.000000,9.000000,0.000000,0.000000,1.000000,0.111111,36.000000,289.000000,3.000000
50%,1.400000e+05,0.000000,5.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,36.000000,961.000000,16.000000,36.000000,1.000000,2.250000,0.444444,81.000000,961.000000,4.000000
75%,1.800000e+05,0.000000,6.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,2.000000,...,121.000000,2601.000000,25.000000,81.000000,4.000000,4.000000,1.777778,132.250000,2601.000000,4.000000
max,2.353477e+06,1.000000,11.000000,1.000000,1.000000,1.000000,1.000000,6.000000,5.000000,8.000000,...,441.000000,9409.000000,169.000000,441.000000,81.000000,36.000000,64.000000,1369.000000,9409.000000,4.000000


In [ ]:
# data standardization with  sklearn
from sklearn.preprocessing import StandardScaler

# copy of datasets
train = df_train.copy()
test = df_test.copy()

# numerical features
SQB_cols = [col for col in train.columns if (col[:3] == "SQB")]
num_cols = SQB_cols + ["v2a1", "agesq"]


# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(train[[i]])
    
    # transform the training data column
    train[i] = scale.transform(train[[i]])
    
    # transform the testing data column
    test[i] = scale.transform(test[[i]])

In [ ]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

cat_numCols = train.columns.difference(num_cols + ["Id", "idhogar", "dependency", "edjefe", "edjefa", "Target"])
print(cat_numCols)

# fit scaler on training data
norm = MinMaxScaler().fit(train[cat_numCols])

# transform training data
train[cat_numCols] = norm.transform(train[cat_numCols])

# transform testing dataabs
test[cat_numCols] = norm.transform(test[cat_numCols])

Index(['abastaguadentro', 'abastaguafuera', 'abastaguano', 'age', 'area1',
       'area2', 'bedrooms', 'cielorazo', 'computer', 'coopele',
       ...
       'techozinc', 'television', 'tipovivi1', 'tipovivi2', 'tipovivi3',
       'tipovivi4', 'tipovivi5', 'v14a', 'v18q', 'v18q1'],
      dtype='object', length=127)


In [ ]:
#output as a csv with columns
train.columns=['Id']+list(codeBook[codeBook.columns[1]])+['Target']
train.to_csv('./data/train_processed.csv', index=False)
train.head()

,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,Males younger than 12 years of age,...,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared,Target
0,ID_279628684,0.302245,0.0,0.2,0.0,1.0,1.0,0.0,0.0,0.0,...,0.335757,0.117871,-0.967066,0.592794,-0.553536,-0.544758,-0.311754,-0.027787,0.117871,4
1,ID_f29eb3ddd,-0.224298,0.0,0.3,0.0,1.0,1.0,1.0,0.0,0.0,...,0.908871,1.634149,-0.967066,1.153720,-0.553536,-0.544758,4.803672,0.442859,1.634149,4
2,ID_68de51c94,0.206510,0.0,0.7,0.0,1.0,1.0,0.0,0.0,0.0,...,0.609289,3.917180,-0.967066,-0.682039,-0.553536,-0.726385,4.803672,0.196840,3.917180,4
3,ID_d671db89c,0.206510,0.0,0.4,0.0,1.0,1.0,1.0,0.0,0.0,...,0.088276,-0.778111,-0.167084,0.860508,0.022340,-0.356403,-0.231825,0.196840,-0.778111,4
4,ID_d56d6f5f5,0.206510,0.0,0.4,0.0,1.0,1.0,1.0,0.0,0.0,...,0.609289,-0.157816,-0.167084,0.860508,0.022340,-0.356403,-0.231825,0.196840,-0.157816,4


In [ ]:
test.columns=['Id']+list(codeBook[codeBook.columns[1]])
test.to_csv('./data/test_processed.csv', index=False)
test.head()

,Id,Monthly rent payment,=1 Overcrowding by bedrooms,number of all rooms in the house,=1 Overcrowding by rooms,=1 has toilet in the household,=1 if the household has refrigerator,owns a tablet,number of tablets household owns,Males younger than 12 years of age,...,Age in years,escolari squared,age squared,hogar_total squared,edjefe squared,hogar_nin squared,overcrowding squared,dependency squared,meaned squared,Age squared
0,ID_2f6873615,2.149930,0.0,0.4,0.0,1.0,1.0,0.0,0.2,0.2,...,0.041237,-0.966775,-0.934908,-0.540409,-0.682039,-0.409567,-0.242045,-0.291772,1.814685,-0.934908
1,ID_1c78846d2,2.149930,0.0,0.4,0.0,1.0,1.0,0.0,0.2,0.2,...,0.422680,2.367707,0.021380,-0.540409,-0.682039,-0.409567,-0.242045,-0.291772,1.814685,0.021380
2,ID_e5442cf6a,2.149930,0.0,0.4,0.0,1.0,1.0,0.0,0.2,0.2,...,0.422680,2.797543,0.021380,-0.540409,-0.682039,-0.409567,-0.242045,-0.291772,1.814685,0.021380
3,ID_a8db26a79,0.278311,0.0,1.3,0.0,1.0,1.0,1.0,0.0,0.0,...,0.608247,2.367707,1.055206,-0.967066,2.581532,-0.553536,-0.544758,-0.311754,1.640867,1.055206
4,ID_a62966799,0.158642,0.0,0.3,0.0,1.0,1.0,1.0,0.0,0.0,...,0.185567,0.609289,-0.758009,-0.967066,-0.682039,-0.409567,-0.726385,4.803672,0.958668,-0.758009


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f92fbbcf-fa92-4caa-ad57-5cd0b8c2766a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>